# Big Data Analytics | End Term Project
Project Title: Text Categorization using NGrams

Developed By: 

    Anshul Khantwal, MT16010
    Ashutosh Mittal, MT16012

In [2]:
#removePunctuationAndLemmatize() method to remove punctuations and make text Lemmatized.
import re
from nltk.stem import PorterStemmer
def removePunctuationAndLemmatize(text):
    stemmer = PorterStemmer()
    text = stemmer.stem(text)
    text = re.sub("[^A-Za-z ]+","",text).lower().strip()
    return text

In [3]:
#Preparing Training and Testing Datas
import glob
baseDirTestData = 'datasets\Test_Data'
baseDirTrainData = 'datasets\Train_Data'
categories = ['Business',"Entertainment","Politics","Sport","Tech"]

TestDir = [baseDirTestData + "\\" + x +"\\*.txt" for x in categories]
TrainDir = [baseDirTrainData + "\\" + x +"\\*.txt" for x in categories]

In [4]:
#Function to generate 3-Grams from text
from nltk.corpus import stopwords
cachedStopWords = stopwords.words("english")

def processTextAndGenerateGrams(text,n):
    text = re.sub(' +',' ',text)
    text = [item for item in text.split(" ") if item not in cachedStopWords]
    words = []
    print(text)
    if len(text)>n:
        for i in range(0,len(text)-n+1):
            words.append(" ".join(text[i:i+n]))
    elif len(text) == n:
        words.append(" ".join(text))
    return words

In [5]:
#processTextAndGenerateGrams("the hello worlds the anshul the mayur",2)

In [6]:
# TODO: Replace <FILL IN> with appropriate code
def wordCount(wordListRDD):
    return wordListRDD.map(lambda x : (x,1)).reduceByKey(lambda x,y:x+y)

In [7]:
def generateNGramsRDD(fileList,ngrams):
    RDD = sc.union([sc.textFile(f,4) for f in fileList])
    RDD = RDD.map(removePunctuationAndLemmatize)
    dataGramsRDD = RDD.flatMap(lambda x: processTextAndGenerateGrams(x.replace("\n",""),ngrams))
    #return dataGramsRDD.map(lambda x : (x,1)).reduceByKey(lambda x,y:x+y)
    return dataGramsRDD.map(lambda x : (x,1)).reduceByKey(lambda x,y:x+y).map(lambda x : (x[0],[x[1]]))

ngrams = 2
NGramsBusiness = generateNGramsRDD(glob.glob(TrainDir[0]),ngrams)
NGramsEntertainment = generateNGramsRDD(glob.glob(TrainDir[1]),ngrams)
NGramsPolitics = generateNGramsRDD(glob.glob(TrainDir[2]),ngrams)
NGramsSport = generateNGramsRDD(glob.glob(TrainDir[3]),ngrams)
NGramsTech = generateNGramsRDD(glob.glob(TrainDir[4]),ngrams)

In [8]:
def generateRepresentativeRDD(NGrams):
    import math
    normalisation = NGrams.map(lambda x : ('*',x[1][0]*x[1][0])).reduceByKey(lambda x,y:x+y)
    sqrt_normalisation = normalisation.map(lambda x: ("*",math.sqrt(x[1])))
    normalising_factor = sqrt_normalisation.values().sum()
    representative_vector = NGrams.map(lambda x : (x[0],[x[1][0]/normalising_factor]))
    return representative_vector

RepresentativeRDDBusiness = generateRepresentativeRDD(NGramsBusiness)
RepresentativeRDDEntertainment = generateRepresentativeRDD(NGramsEntertainment)
RepresentativeRDDPolitics = generateRepresentativeRDD(NGramsPolitics)
RepresentativeRDDSport = generateRepresentativeRDD(NGramsSport)
RepresentativeRDDTech = generateRepresentativeRDD(NGramsTech)

In [9]:
def combineFilesIntoSingleFile(dir_list,filename):
    c = ""
    for f in glob.glob(TrainDir[4]):
        with open(f) as f:
            a = f.read()
            c+=a
            
    with open(filename,"w") as f:
        f.write(c)

In [10]:
#dir_list = glob.glob(TrainDir[4])
#combineFilesIntoSingleFile(glob.glob(TrainDir[4]),"abc.txt")
#RepresentativeRDDPolitics.takeOrdered(10,lambda x:-x[1][0])

In [11]:
def findDistance(NormalisedRDD,RepresentativeRDD):
    cosineDistance = NormalisedRDD.union(RepresentativeRDD).reduceByKey(lambda x,y:x+y)
    #print(cosineDistance.filter(lambda x:len(x[1])!=1).take(20))
    return cosineDistance.filter(lambda x:len(x[1])!=1).map(lambda x:("*",x[1][0]*x[1][1])).values().sum()

In [12]:
def classifyUsingCosine(NormalisedRDD):
    similarity = 0
    label = None
    
    x = findDistance(NormalisedRDD,RepresentativeRDDBusiness)
    if similarity < x:
        similarity = x
        label = "Business"
        
    x = findDistance(NormalisedRDD,RepresentativeRDDEntertainment)
    if similarity < x:
        similarity = x
        label = "Entertainment"
        
    x = findDistance(NormalisedRDD,RepresentativeRDDPolitics)
    if similarity < x:
        similarity = x
        label = "Politics"
        
    x = findDistance(NormalisedRDD,RepresentativeRDDSport)
    if similarity < x:
        similarity = x
        label = "Sport"
        
    x = findDistance(NormalisedRDD,RepresentativeRDDTech)
    if similarity < x:
        similarity = x
        label = "Technology"
        
    return (similarity,label)

In [13]:
def findEuclideanDistance(NormalisedRDD,RepresentativeRDD):
    import math
    distance = NormalisedRDD.union(RepresentativeRDD).reduceByKey(lambda x,y:x+y)
    #print(cosineDistance.filter(lambda x:len(x[1])!=1).take(20))
    return math.sqrt( distance.map( lambda x : ("*",math.pow(x[1][0]-x[1][1],2)) if len(x[1]) == 2 else ("*",math.pow(x[1][0],2)) ).values().sum() )

In [14]:
def classifyUsingEuclidean(NormalisedRDD):
    similarity = 2
    label = None
    
    x = findEuclideanDistance(NormalisedRDD,RepresentativeRDDBusiness)
    if similarity > x:
        similarity = x
        label = "Business"
        
    x = findEuclideanDistance(NormalisedRDD,RepresentativeRDDEntertainment)
    if similarity > x:
        similarity = x
        label = "Entertainment"
        
    x = findEuclideanDistance(NormalisedRDD,RepresentativeRDDPolitics)
    if similarity > x:
        similarity = x
        label = "Politics"
        
    x = findEuclideanDistance(NormalisedRDD,RepresentativeRDDSport)
    if similarity > x:
        similarity = x
        label = "Sport"
        
    x = findEuclideanDistance(NormalisedRDD,RepresentativeRDDTech)
    if similarity > x:
        similarity = x
        label = "Technology"
        
    return (similarity,label)

In [16]:
testing_data = [["business.txt"],["entertainment.txt"],["politics.txt"],["sports.txt"],["tech.txt"]]
NGramsTest = generateNGramsRDD(testing_data[4],ngrams)
NormalisedRDD = generateRepresentativeRDD(NGramsTest)

In [17]:
print("Statistics while using Cosine Distance as a norm:\t",classifyUsingCosine(NormalisedRDD))
print("Statistics while using Euclidean Distance as a norm:\t",classifyUsingEuclidean(NormalisedRDD))

Statistics while using Cosine Distance as a norm:	 (0.05850126496138351, 'Technology')
Statistics while using Euclidean Distance as a norm:	 (1.3722235496001984, 'Technology')
